In [1]:
pip install pyspark

In [2]:
# Yonghwan Kim A11746276
import matplotlib.pyplot as plt
import numpy as np
import sys
# spark context intialized
from pyspark import SparkContext, SparkConf
sc = SparkContext("local","assignment7")

In [3]:
# mount the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# directory to the input file
graph_dataset_dir = '/content/drive/MyDrive/email-Eu-core.txt'

In [5]:
# import the graph input file to RDD
rddTextfile = sc.textFile(graph_dataset_dir)
# collect node data
nodeRDD = rddTextfile.flatMap(lambda x: x.split()).distinct().map(lambda x: int(x))
# collect edge data
edgeRDD = rddTextfile.map(lambda x: x.split()).map(lambda x: (int(x[0]), int(x[1])))
edgeList = edgeRDD.collect()
# number of nodes
nodeNum = nodeRDD.count()
# number of edges
edgeNum = edgeRDD.count()
print(nodeNum, edgeNum)
# funtion to put edge info to L matrix
def L_mt(L_np):
    for i, j in edgeList:
        L_np[i][j] = 1
    return L_np
# Create L matrix initialize with zeros and put edge information later
L_RDD = sc.parallelize(L_mt(np.zeros((nodeNum,nodeNum))))
# L transpose matrix
LT_RDD = sc.parallelize(np.transpose(L_mt(np.zeros((nodeNum,nodeNum)))))
# hub column vector with elements having 1
h = np.ones((nodeNum)).reshape(nodeNum, 1)
# authority column vector with elements having 0
a = np.zeros((nodeNum)).reshape(nodeNum, 1)
# iterate 40 times until converge
for r in range(40):
    # authority vector is dot product of L transpose matrix and h vector
    a_rdd = LT_RDD.map(lambda x: np.dot(x, h))
    a = a_rdd.collect()
    # hub vector is dot product of L matrix and a vector
    h_rdd = L_RDD.map(lambda x: np.dot(x, a))
    h = h_rdd.collect()
    # normalize authority and hub vectors to scale between 0 to 1
    normalize_a = np.sqrt(sum([x**2 for x in a]))
    normalize_h = np.sqrt(sum([x**2 for x in h]))
    a = [x / normalize_a for x in a]
    h = [x / normalize_h for x in h]

    # print("a: ", list(np.around(a, 3)))
    # print("h: ", list(np.around(h, 3)))

1005 25571


In [6]:
h_list = []
h_counter = 0
# indexing
for h1 in h:
    h_list.append((h_counter, h1))
    h_counter = h_counter + 1
# sorted in decreasing order to get the top 5 hub scores
h_reverse_sorted = sorted(h_list, key = lambda x: x[1], reverse = True)
print("List of 5 top node ids and their hub scores")
print(h_reverse_sorted[0], h_reverse_sorted[1], h_reverse_sorted[2], h_reverse_sorted[3], h_reverse_sorted[4])
print("\n")
# sorted in increasing order to get the buttom 5 hub scores
h_sorted = sorted(h_list, key = lambda x: x[1], reverse = False)
print("List of 5 bottom node ids and their hub scores")
print(h_sorted[0], h_sorted[1], h_sorted[2], h_sorted[3], h_sorted[4])
print("\n")

a_list = []
a_counter = 0
# indexing
for a1 in a:
    a_list.append((a_counter, a1))
    a_counter = a_counter + 1
# sorted in decreasing order to get the top 5 authority scores
a_reverse_sorted = sorted(a_list, key = lambda x: x[1], reverse = True)
print("List of 5 top node ids and their authority scores")
print(a_reverse_sorted[0], a_reverse_sorted[1], a_reverse_sorted[2], a_reverse_sorted[3], a_reverse_sorted[4])
print("\n")
# sorted in increasing order to get the buttom 5 authority scores
a_sorted = sorted(a_list, key = lambda x: x[1], reverse = False)
print("List of 5 bottom node ids and their authority scores")
print(a_sorted[0], a_sorted[1], a_sorted[2], a_sorted[3], a_sorted[4])

List of 5 top node ids and their hub scores
(160, array([0.19155185])) (82, array([0.17331116])) (121, array([0.17175556])) (107, array([0.15837819])) (62, array([0.14836754]))


List of 5 bottom node ids and their hub scores
(78, array([0.])) (203, array([0.])) (239, array([0.])) (289, array([0.])) (315, array([0.]))


List of 5 top node ids and their authority scores
(160, array([0.14388814])) (107, array([0.13746519])) (62, array([0.13343406])) (434, array([0.12923347])) (121, array([0.12896424]))


List of 5 bottom node ids and their authority scores
(524, array([0.])) (750, array([0.])) (755, array([0.])) (790, array([0.])) (858, array([0.]))


List of 5 top node ids and their hub scores
(160, array([0.19155185])) (82, array([0.17331116])) (121, array([0.17175556])) (107, array([0.15837819])) (62, array([0.14836754]))


List of 5 bottom node ids and their hub scores
(78, array([0.])) (203, array([0.])) (239, array([0.])) (289, array([0.])) (315, array([0.]))


List of 5 top node ids and their authority scores
(160, array([0.14388814])) (107, array([0.13746519])) (62, array([0.13343406])) (434, array([0.12923347])) (121, array([0.12896424]))


List of 5 bottom node ids and their authority scores
(524, array([0.])) (750, array([0.])) (755, array([0.])) (790, array([0.])) (858, array([0.]))